# Proyecto Final Tecnologías Emergentes

### Integrantes
    Fabio Buitrago
    Luis Alejandro Bravo Ferreira
    Ana María Ortegón Sepúlveda

### Objetivo
El objetivo del proyecto es resolver un problema contextualizado en sistemas de información geográfica realizando una clasificación de imágenes satelitales. Para esto se debe hacer uso del conjunto de imágenes adjunta a este proyecto, explorando los datos, codificando el modelo deseado y generando un reporte con los resultados de la solución construida.

## Descripción del problema

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import time
import os
import shutil
from sklearn.model_selection import train_test_split


from PIL import Image
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score


from tensorflow.keras.preprocessing.image import load_img, img_to_array

## Exploración y Descripción del conjunto de datos

In [ ]:
"""
    Analiza y resume los datos de imágenes en diferentes categorías.

    La función itera sobre cada categoría especificada, accede a su correspondiente
    directorio dentro del directorio base y procesa todas las imágenes contenidas en él.
    Para cada imagen, calcula el promedio y desviación estándar de los colores, así como
    su histograma de color. Cada detalle de imagen es agregado a un DataFrame para
    posterior análisis. En caso de que no exista el directorio de alguna categoría, se
    maneja la excepción correspondiente.

    Args:
    - base_dir (str): Directorio base que contiene las carpetas de cada categoría.
    - categories (list of str): Lista de nombres de categorías para procesar.

    Returns:
    - dict: Un diccionario que contiene un resumen de datos para cada categoría, incluyendo
            un DataFrame con los detalles de las imágenes y el total de imágenes en esa categoría.
"""
def summarize_image_data(base_dir, categories):

    data_summary = {}
    for category in categories:
        category_path = os.path.join(base_dir, category)
        try:
            images = os.listdir(category_path)
            image_details = []
            for image_name in images:
                image_path = os.path.join(category_path, image_name)
                with Image.open(image_path) as img:
                    img_array = np.array(img)
                    mean_colors = np.mean(img_array, axis=(0, 1)) if img_array.ndim == 3 else np.mean(img_array)
                    std_colors = np.std(img_array, axis=(0, 1)) if img_array.ndim == 3 else np.std(img_array)
                    hist_colors = [np.histogram(img_array[:, :, i], bins=256, range=(0, 256))[0]
                                   for i in range(img_array.shape[-1])] if img_array.ndim == 3 else np.histogram(img_array, bins=256, range=(0, 256))[0]
                    image_details.append({
                        'filename': image_name,
                        'width': img.width,
                        'height': img.height,
                        'mode': img.mode,
                        'mean_colors': mean_colors,
                        'std_colors': std_colors,
                        'histogram': hist_colors
                    })
            data_summary[category] = {
                'count': len(images),
                'details': pd.DataFrame(image_details)
            }
        except FileNotFoundError:
            print(f"No se encontró la carpeta: {category_path}")
    return data_summary

def show_sample_images_and_histograms(data_summary, base_dir, categories):
    fig, axs = plt.subplots(2, len(categories), figsize=(20, 10), gridspec_kw={'height_ratios': [3, 1]})
    for i, category in enumerate(categories):
        if data_summary.get(category):
            image_path = os.path.join(base_dir, category, data_summary[category]['details'].iloc[0]['filename'])
            img = Image.open(image_path)
            img_array = np.array(img)
            axs[0, i].imshow(img)
            axs[0, i].set_title(f"{category} (Muestra)")
            axs[0, i].axis('off')

            if img_array.ndim == 3:
                for j in range(3):
                    axs[1, i].plot(data_summary[category]['details'].iloc[0]['histogram'][j], color=['r', 'g', 'b'][j])
            else:
                axs[1, i].plot(data_summary[category]['details'].iloc[0]['histogram'], color='gray')
    plt.show()

base_dir = 'data'
categories = ['cloudy', 'green_area', 'desert', 'water']

data_summary = summarize_image_data(base_dir, categories)
show_sample_images_and_histograms(data_summary, base_dir, categories)

for category in categories:
    if data_summary.get(category):
        print(f"Categoría: {category}")
        print(f"  Número total de imágenes: {data_summary[category]['count']}")
        print(f"  Media de colores: {data_summary[category]['details']['mean_colors'].mean()}")
        print(f"  Desviación estándar de colores: {data_summary[category]['details']['std_colors'].mean()}")


## Procedimiento

In [ ]:
"""
    Divide las imágenes de una categoría en conjuntos de entrenamiento y prueba,
    y copia las imágenes correspondientes en los directorios respectivos.

    Args:
        category (str): El nombre de la categoría (subdirectorio) de las imágenes.
        source_dir (str): La ruta del directorio fuente que contiene las imágenes.
        train_dir (str): La ruta del directorio principal para las imágenes de entrenamiento.
        test_dir (str): La ruta del directorio principal para las imágenes de prueba.
        train_size (float, optional): Proporción de las imágenes que se utilizarán para el entrenamiento. 
                                      El resto se utilizará para la prueba. Por defecto es 0.8.
"""
def split_and_copy_images(category, source_dir, train_dir, test_dir, train_size=0.8):
    images = os.listdir(source_dir)
    train_images, test_images = train_test_split(images, train_size=train_size, random_state=42)
    
    category_train_dir = os.path.join(train_dir, category)
    category_test_dir = os.path.join(test_dir, category)
    
    os.makedirs(category_train_dir, exist_ok=True)
    os.makedirs(category_test_dir, exist_ok=True)
    
    for image in train_images:
        src_path = os.path.join(source_dir, image)
        dst_path = os.path.join(category_train_dir, image)
        shutil.copy(src_path, dst_path)
    
    for image in test_images:
        src_path = os.path.join(source_dir, image)
        dst_path = os.path.join(category_test_dir, image)
        shutil.copy(src_path, dst_path)

"""
    Prepara los generadores de datos de imágenes para entrenamiento y prueba.

    Args:
        cloudy_path (str): Ruta del directorio que contiene las imágenes de la categoría 'cloudy'.
        desert_path (str): Ruta del directorio que contiene las imágenes de la categoría 'desert'.
        green_path (str): Ruta del directorio que contiene las imágenes de la categoría 'green'.
        water_path (str): Ruta del directorio que contiene las imágenes de la categoría 'water'.
        base_dir (str, optional): Ruta base donde se crearán los directorios de entrenamiento y prueba. 
                                  Por defecto es './data/'.
        train_size (float, optional): Proporción de las imágenes que se utilizarán para el entrenamiento. 
                                      El resto se utilizará para la prueba. Por defecto es 0.8.
        new_img_width (int, optional): Ancho al que se redimensionarán las imágenes. Por defecto es 64.
        new_img_height (int, optional): Altura a la que se redimensionarán las imágenes. Por defecto es 64.
        batch_size (int, optional): Tamaño del lote para los generadores. Por defecto es 32.

    Returns:
        tuple: Generadores de datos de imágenes para entrenamiento y prueba.
"""
def prepare_data_generators(cloudy_path, desert_path, green_path, water_path, base_dir='./data/', train_size=0.8, new_img_width=64, new_img_height=64, batch_size=32):
    # Definir rutas para los datasets divididos
    train_dir = os.path.join(base_dir, 'training_set')
    test_dir = os.path.join(base_dir, 'test_set')

    # Crear directorios para entrenamiento y prueba
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Lista de categorías y sus rutas correspondientes
    categories = {
        'cloudy': cloudy_path,
        'desert': desert_path,
        'green': green_path,
        'water': water_path
    }

    # Dividir y copiar imágenes para cada categoría
    for category, path in categories.items():
        split_and_copy_images(category, path, train_dir, test_dir, train_size)

    # Crear generadores de datos de imágenes para entrenamiento y prueba
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    test_datagen = ImageDataGenerator(rescale=1./255)

    # Flujo desde el directorio y redimensionamiento de las imágenes a (64, 64)
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(new_img_width, new_img_height),
        batch_size=batch_size,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(new_img_width, new_img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, test_generator

# Dimensiones de las imágenes
new_img_width=64
new_img_height=64

# Tamaño del lote
batch_size = 32

# Definir rutas de los datasets originales
cloudy_path = "./data/cloudy"
desert_path = "data/desert"
green_path = "data/green_area"
water_path = "data/water"

# Uso de la función
train_generator, test_generator = prepare_data_generators(cloudy_path, desert_path, green_path, water_path, batch_size = batch_size)

## Arquitectura #1

In [ ]:
"""
    Define y compila un modelo CNN para la clasificación de imágenes.

    Args:
        input_shape (tuple, optional): Forma de las imágenes de entrada. Por defecto es (64, 64, 3).
        num_classes (int, optional): Número de clases de salida. Por defecto es 4.
        learning_rate (float, optional): Tasa de aprendizaje para el optimizador. Por defecto es 0.00001.

    Returns:
        Model: Modelo compilado de Keras.
"""
def create_cnn_model(input_shape=(64, 64, 3), num_classes=4, learning_rate=0.00001):
    
    inputs = Input(shape=input_shape)
    
    x = Conv2D(32, (3, 3), activation='relu', padding="same")(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    
    x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    
    x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    
    x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    
    x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    custom_optimizer = Adam(learning_rate=learning_rate)

    # Compilar el modelo
    model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Mostrar el resumen del modelo
    model.summary()
    
    return model

# Uso de la función
model = create_cnn_model()

## Arquitectura #2

In [ ]:
def create_cnn_model_v2(input_shape=(64, 64, 3), num_classes=4, learning_rate=0.00001):
    """
    Define y compila un segundo modelo CNN para la clasificación de imágenes.

    Args:
        input_shape (tuple, optional): Forma de las imágenes de entrada. Por defecto es (64, 64, 3).
        num_classes (int, optional): Número de clases de salida. Por defecto es 4.
        learning_rate (float, optional): Tasa de aprendizaje para el optimizador. Por defecto es 0.00001.

    Returns:
        Model: Modelo compilado de Keras.
    """
    inputs = Input(shape=input_shape)
    
    x = Conv2D(32, (3, 3), activation='relu', padding="same")(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    
    x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    
    x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    
    x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)

    x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    
    x = Conv2D(1024, (3, 3), activation='relu', padding="same")(x)
    x = Conv2D(1024, (3, 3), activation='relu', padding="same")(x)
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    custom_optimizer = Adam(learning_rate=learning_rate)

    # Compilar el modelo
    model.compile(optimizer=custom_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Mostrar el resumen del modelo
    model.summary()
    
    return model

# Uso de la función
model_v2 = create_cnn_model_v2()


### Función para matrices de confusión y entrenamiento

In [ ]:
"""
    Evalúa el modelo CNN utilizando el generador de datos de prueba, calcula varias métricas de rendimiento,
    y genera una matriz de confusión visualmente atractiva.

    Args:
        model (Model): Modelo de Keras entrenado.
        test_generator (DirectoryIterator): Generador de datos de prueba.
        train_generator (DirectoryIterator): Generador de datos de entrenamiento (usado para obtener índices de clase).

    Returns:
        dict: Diccionario que contiene la matriz de confusión, el informe de clasificación y métricas adicionales (precisión, recall, F1 score).
"""
def evaluate_and_plot_confusion_matrix(model, test_generator, train_generator):
    
    # Reiniciar el generador de prueba
    test_generator.reset()
    
    # Realizar predicciones sobre el conjunto de prueba
    predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)
    
    # Obtener las etiquetas predichas y verdaderas
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = test_generator.classes
    
    # Obtener los índices de clase y nombres de clase
    class_indices = train_generator.class_indices
    class_names = list(class_indices.keys())
    
    # Calcular la matriz de confusión
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    
    # Generar el heatmap de la matriz de confusión
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Matriz de Confusión')
    plt.xlabel('Etiqueta Predicha')
    plt.ylabel('Etiqueta Verdadera')
    plt.show()
    
    # Calcular el informe de clasificación
    class_report = classification_report(true_labels, predicted_labels, target_names=class_names)
    print('Informe de Clasificación')
    print(class_report)
    
    # Calcular métricas adicionales
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='weighted', zero_division=1)
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    
    print(f'Precisión: {accuracy}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    
    # Retornar las métricas calculadas en un diccionario
    return {
        'confusion_matrix': conf_matrix,
        'classification_report': class_report,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }


"""
    Entrena el modelo CNN con los generadores de datos de entrenamiento y prueba.

    Args:
        model (Model): Modelo de Keras a entrenar.
        train_generator (DirectoryIterator): Generador de datos de entrenamiento.
        test_generator (DirectoryIterator): Generador de datos de prueba.
        epochs (int, optional): Número de épocas para entrenar el modelo. Por defecto es 10.

    Returns:
        tuple: Historia del entrenamiento y tiempo total de entrenamiento en segundos.
"""
def train_cnn_model(model, train_generator, test_generator, epochs=10):

    start_time = time.time()
    
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=test_generator
    )
    
    end_time = time.time()
    total_time = end_time - start_time
    
    return history, total_time

## Modelo 1

### Entrenamiento del modelo 1 por 10 épocas

In [ ]:
# Uso de la función
history, training_time = train_cnn_model(model, train_generator, test_generator, epochs=10)

# Mostrar el tiempo total de entrenamiento
print(f"Tiempo total de entrenamiento: {training_time} segundos")

In [ ]:
# Uso de la función
evaluation_metrics = evaluate_and_plot_confusion_matrix(model, test_generator, train_generator)

# Acceso a las métricas
conf_matrix = evaluation_metrics['confusion_matrix']
class_report = evaluation_metrics['classification_report']
accuracy = evaluation_metrics['accuracy']
precision = evaluation_metrics['precision']
recall = evaluation_metrics['recall']
f1 = evaluation_metrics['f1_score']

### Entrenamiento del modelo 1 por 50 épocas

In [ ]:
# Uso de la función
history, training_time = train_cnn_model(model, train_generator, test_generator, epochs=50)

# Mostrar el tiempo total de entrenamiento
print(f"Tiempo total de entrenamiento: {training_time} segundos")

In [ ]:
# Uso de la función
evaluation_metrics = evaluate_and_plot_confusion_matrix(model, test_generator, train_generator)

# Acceso a las métricas
conf_matrix = evaluation_metrics['confusion_matrix']
class_report = evaluation_metrics['classification_report']
accuracy = evaluation_metrics['accuracy']
precision = evaluation_metrics['precision']
recall = evaluation_metrics['recall']
f1 = evaluation_metrics['f1_score']

### Entrenamiento del modelo 1 por 100 épocas

In [ ]:
# Uso de la función
history, training_time = train_cnn_model(model, train_generator, test_generator, epochs=100)

# Mostrar el tiempo total de entrenamiento
print(f"Tiempo total de entrenamiento: {training_time} segundos")

In [ ]:
# Uso de la función
evaluation_metrics = evaluate_and_plot_confusion_matrix(model, test_generator, train_generator)

# Acceso a las métricas
conf_matrix = evaluation_metrics['confusion_matrix']
class_report = evaluation_metrics['classification_report']
accuracy = evaluation_metrics['accuracy']
precision = evaluation_metrics['precision']
recall = evaluation_metrics['recall']
f1 = evaluation_metrics['f1_score']

## Modelo 2

### Entrenamiento del modelo 2 por 10 épocas

In [ ]:
# Uso de la función
history_v2, training_time_v2 = train_cnn_model(model_v2, train_generator, test_generator, epochs=10)

# Mostrar el tiempo total de entrenamiento
print(f"Tiempo total de entrenamiento: {training_time_v2} segundos")

In [ ]:
# Uso de la función
evaluation_metrics = evaluate_and_plot_confusion_matrix(model_v2, test_generator, train_generator)

# Acceso a las métricas
conf_matrix = evaluation_metrics['confusion_matrix']
class_report = evaluation_metrics['classification_report']
accuracy = evaluation_metrics['accuracy']
precision = evaluation_metrics['precision']
recall = evaluation_metrics['recall']
f1 = evaluation_metrics['f1_score']

### Entrenamiento del modelo 2 por 50 épocas

In [ ]:
# Uso de la función
history_v2, training_time_v2 = train_cnn_model(model_v2, train_generator, test_generator, epochs=50)

# Mostrar el tiempo total de entrenamiento
print(f"Tiempo total de entrenamiento: {training_time_v2} segundos")

In [ ]:
# Uso de la función
evaluation_metrics = evaluate_and_plot_confusion_matrix(model_v2, test_generator, train_generator)

# Acceso a las métricas
conf_matrix = evaluation_metrics['confusion_matrix']
class_report = evaluation_metrics['classification_report']
accuracy = evaluation_metrics['accuracy']
precision = evaluation_metrics['precision']
recall = evaluation_metrics['recall']
f1 = evaluation_metrics['f1_score']

### Entrenamiento del modelo 2 por 100 épocas

In [ ]:
# Uso de la función
history_v2, training_time_v2 = train_cnn_model(model_v2, train_generator, test_generator, epochs=100)

# Mostrar el tiempo total de entrenamiento
print(f"Tiempo total de entrenamiento: {training_time_v2} segundos")

In [ ]:
# Uso de la función
evaluation_metrics = evaluate_and_plot_confusion_matrix(model_v2, test_generator, train_generator)

# Acceso a las métricas
conf_matrix = evaluation_metrics['confusion_matrix']
class_report = evaluation_metrics['classification_report']
accuracy = evaluation_metrics['accuracy']
precision = evaluation_metrics['precision']
recall = evaluation_metrics['recall']
f1 = evaluation_metrics['f1_score']

# Ver ejemplos de instancias mal clasificadas

In [ ]:
"""
    Muestra las imágenes mal clasificadas junto con sus etiquetas verdaderas y predichas.

    Args:
        model (Model): El modelo de Keras entrenado.
        test_generator (DirectoryIterator): Generador de datos de prueba.
        class_names (list): Lista de nombres de clases.
        num_images (int): Número de imágenes mal clasificadas a mostrar. Por defecto son 20.
"""
def display_misclassified_images(model, test_generator, class_names, num_images=20):

    # Reiniciar el generador de prueba
    test_generator.reset()
    
    # Realizar predicciones sobre el conjunto de prueba
    predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)
    
    # Obtener las etiquetas predichas y verdaderas
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = test_generator.classes
    
    # Encontrar índices de imágenes mal clasificadas
    misclassified_indices = np.where(predicted_labels != true_labels)[0]
    
    # Mostrar algunas imágenes mal clasificadas
    num_display = min(num_images, len(misclassified_indices))
    plt.figure(figsize=(15, num_display * 2))
    
    for i, idx in enumerate(misclassified_indices[:num_display]):
        # Obtener ruta de la imagen
        img_path = test_generator.filepaths[idx]
        
        # Cargar la imagen
        img = load_img(img_path, target_size=(test_generator.target_size))
        img_array = img_to_array(img)
        
        # Mostrar la imagen
        plt.subplot(num_display, 1, i + 1)
        plt.imshow(img_array.astype("uint8"))
        plt.axis('off')
        
        # Etiquetas verdadera y predicha
        true_label = class_names[true_labels[idx]]
        predicted_label = class_names[predicted_labels[idx]]
        plt.title(f'True: {true_label} | Predicted: {predicted_label}')
    
    plt.tight_layout()
    plt.show()

# Uso de la función
class_names = list(train_generator.class_indices.keys())
display_misclassified_images(model, test_generator, class_names)


## Conclusiones